In [14]:
# !pip install python-binance
# !pip install python-dotenv
# !pip install pandas_ta

In [28]:
from binance.client import Client
import pandas as pd
from dotenv import load_dotenv 
import os
import pandas_ta as ta
from sklearn.preprocessing import StandardScaler

In [38]:
from sklearn.model_selection import train_test_split # Para dividir os dados
from sklearn.ensemble import RandomForestClassifier # O algoritmo de Floresta Aleatória
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score # Métricas de avaliação


In [3]:
load_dotenv()

API_KEY = os.getenv("API_KEY")
API_SECRET = os.getenv("API_SECRET")

In [32]:
if not API_KEY or not API_SECRET:
    print("Erro: As chaves API_KEY ou API_SECRET não foram carregadas do arquivo .env.")
    print("Verifique se o arquivo .env existe na mesma pasta e se as variáveis estão corretas.")
    exit() # Sai do programa se as chaves não forem encontradas

# 4. Inicializa o cliente da Binance (mesmo código de antes)
try:
    client = Client(API_KEY, API_SECRET)
    print("Conexão com a Binance estabelecida com sucesso!")
except Exception as e:
    print(f"Erro ao conectar com a Binance: {e}")
    print("Verifique suas chaves de API e sua conexão com a internet.")
    exit()

# 5. Define os parâmetros para a busca dos dados de candlestick (mesmo código de antes)
symbol = 'BTCUSDT'
interval = '1h'
limit = 2500

print(f"\nBuscando os últimos {limit} candlesticks de {symbol} no intervalo de {interval}...")


Conexão com a Binance estabelecida com sucesso!

Buscando os últimos 2500 candlesticks de BTCUSDT no intervalo de 1h...


In [5]:
# 6. Faz a requisição para a API da Binance
klines = client.get_historical_klines(symbol, interval, limit=limit)


In [33]:
# 7. Processa os dados brutos e os organiza em um DataFrame do Pandas
data = []
for kline in klines:
    data.append({
        'timestamp': kline[0],
        'open': float(kline[1]),
        'high': float(kline[2]),
        'low': float(kline[3]),
        'close': float(kline[4]),
        'volume': float(kline[5])
    })

df = pd.DataFrame(data)
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# Armazenar o DataFrame original antes de renomear colunas
# para que possamos usar 'close' para o cálculo do label
df_original_cols = df.copy()

# Assegura que as colunas numéricas estão no tipo correto para pandas_ta
df.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

In [18]:
# --- 8. Adicionar Indicadores Técnicos (Feature Engineering) ---
print("\nAdicionando indicadores técnicos ao DataFrame...")

# Média Móvel Simples (SMA) de 20 períodos
# df.ta.sma(length=20, append=True) adiciona a coluna 'SMA_20' diretamente ao DataFrame
df.ta.sma(length=20, append=True)

# Bandas de Bollinger (BBands)
# bb_df = df.ta.bbands(append=True) adiciona várias colunas: 'BBL_5_2.0', 'BBM_5_2.0', 'BBU_5_2.0', etc.
df.ta.bbands(append=True)

# Índice de Força Relativa (RSI) de 14 períodos
df.ta.rsi(length=14, append=True)

print("Indicadores adicionados com sucesso!")


Adicionando indicadores técnicos ao DataFrame...
Indicadores adicionados com sucesso!


In [25]:
# --- 9 Tratamento de Valores NaN ---
print("\nTratando valores NaN...")

# Guarda o número de linhas antes de remover os NaNs
initial_rows = len(df)

# Remove todas as linhas que contêm qualquer valor NaN
df.dropna(inplace=True)

# Guarda o número de linhas depois de remover os NaNs
final_rows = len(df)

print(f"Número de linhas antes do tratamento de NaN: {initial_rows}")
print(f"Número de linhas após o tratamento de NaN: {final_rows}")
print(f"Total de linhas removidas: {initial_rows - final_rows}")


Tratando valores NaN...
Número de linhas antes do tratamento de NaN: 500
Número de linhas após o tratamento de NaN: 481
Total de linhas removidas: 19


In [29]:
### **10. Normalização/Escalonamento de Dados (StandardScaler)**
print("\nRealizando o escalonamento dos dados (StandardScaler)...")
# Certifique-se de que todas as colunas sejam numéricas antes de escalonar
features_to_scale = df.columns.tolist()
# Cria uma instância do StandardScaler
scaler = StandardScaler()
# O resultado é um array NumPy, então o convertemos de volta para um DataFrame.
df_scaled = pd.DataFrame(scaler.fit_transform(df[features_to_scale]),
                         columns=features_to_scale,
                         index=df.index)

print("Escalonamento concluído!")


Realizando o escalonamento dos dados (StandardScaler)...
Escalonamento concluído!


In [39]:
# --- 10. Criação dos Labels (Comprar/Vender/Manter) ---
print("\nCriando os labels de Comprar/Vender/Manter...")

look_forward_period = 5
price_change_threshold = 0.005

df['future_close'] = df['Close'].shift(-look_forward_period)
df['price_change'] = (df['future_close'] - df['Close']) / df['Close']

def get_label(change, threshold):
    if change > threshold:
        return 1  # Comprar
    elif change < -threshold:
        return -1 # Vender
    else:
        return 0  # Manter

df['label'] = df['price_change'].apply(lambda x: get_label(x, price_change_threshold))

df.drop(columns=['future_close', 'price_change'], inplace=True)
df.dropna(inplace=True) # Remove as últimas linhas que não têm um 'future_close'

print("Labels criados com sucesso!")


Criando os labels de Comprar/Vender/Manter...
Labels criados com sucesso!


In [40]:
# 11. Dividir em Features (X) e Labels (y) e aplicar escalonamento
X = df.drop(columns=['label'])
y = df['label']

# Agora, escalonar as features (X)
features_to_scale_final = X.columns.tolist()
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X[features_to_scale_final]),
                        columns=features_to_scale_final,
                        index=X.index)

# Verificação final de NaNs após todo o pré-processamento
if X_scaled.isnull().sum().sum() > 0 or y.isnull().sum() > 0:
    print("AVISO: Ainda existem NaNs em X_scaled ou y após o pré-processamento. Verifique!")
    print(X_scaled.isnull().sum())
    print(y.isnull().sum())
    exit() # Interrompe o processo se houver NaNs restantes

In [41]:
# --- 12. Divisão dos Dados em Conjuntos de Treino e Teste ---
print("\nDividindo os dados em conjuntos de treino e teste...")

# test_size=0.20 significa que 20% dos dados serão para teste, 80% para treino.
# random_state=42 garante que a divisão seja a mesma toda vez que você rodar o código,
# o que é bom para reprodutibilidade.
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.20, random_state=42, stratify=y)
# stratify=y é importante para problemas de classificação desbalanceados,
# garantindo que a proporção de cada classe (1, 0, -1) seja mantida tanto no treino quanto no teste.

print(f"Tamanho do conjunto de treino (X_train): {len(X_train)} linhas")
print(f"Tamanho do conjunto de teste (X_test): {len(X_test)} linhas")


Dividindo os dados em conjuntos de treino e teste...
Tamanho do conjunto de treino (X_train): 400 linhas
Tamanho do conjunto de teste (X_test): 100 linhas


In [42]:
# --- 13. Treinamento do Modelo de Classificação (Random Forest) ---
print("\nIniciando o treinamento do modelo RandomForestClassifier...")

# Cria uma instância do modelo RandomForestClassifier
# n_estimators: número de árvores na floresta (mais árvores geralmente melhor, mas mais lento)
# random_state: para reprodutibilidade
# class_weight: importante para classes desbalanceadas. 'balanced' tenta ajustar pesos inversamente proporcionais
# à frequência das classes.
model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

# Treina o modelo com os dados de treino
model.fit(X_train, y_train)

print("Treinamento do modelo concluído com sucesso!")


Iniciando o treinamento do modelo RandomForestClassifier...
Treinamento do modelo concluído com sucesso!


In [43]:
# --- 14. Avaliação do Modelo ---
print("\nIniciando a avaliação do modelo...")

# Faz previsões no conjunto de teste
y_pred = model.predict(X_test)

# Calcula a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAcurácia do modelo no conjunto de teste: {accuracy:.4f}")

# Exibe o relatório de classificação (precisão, recall, f1-score para cada classe)
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# Exibe a matriz de confusão
print("\nMatriz de Confusão:")
# A matriz mostra:
# Linhas: Valores Reais (True Labels)
# Colunas: Valores Previstos (Predicted Labels)
# Ex: Célula (0, 1) = Quantas vezes o modelo previu '1' (Comprar) quando o real era '0' (Manter)
cm = confusion_matrix(y_test, y_pred)
print(cm)



Iniciando a avaliação do modelo...

Acurácia do modelo no conjunto de teste: 0.6400

Relatório de Classificação:
              precision    recall  f1-score   support

          -1       0.60      0.38      0.46        16
           0       0.70      0.82      0.75        65
           1       0.36      0.26      0.30        19

    accuracy                           0.64       100
   macro avg       0.55      0.48      0.51       100
weighted avg       0.62      0.64      0.62       100


Matriz de Confusão:
[[ 6  9  1]
 [ 4 53  8]
 [ 0 14  5]]
